In [1]:
!pip3 install transformers

  Obtaining dependency information for transformers from https://files.pythonhosted.org/packages/12/dd/f17b11a93a9ca27728e12512d167eb1281c151c4c6881d3ab59eb58f4127/transformers-4.35.2-py3-none-any.whl.metadata
  Using cached transformers-4.35.2-py3-none-any.whl.metadata (123 kB)
  Obtaining dependency information for huggingface-hub<1.0,>=0.16.4 from https://files.pythonhosted.org/packages/05/09/1945ca6ba3ad8ad6e2872ba682ce8d68c5e63c8e55458ed8ab4885709f1d/huggingface_hub-0.19.4-py3-none-any.whl.metadata
  Using cached huggingface_hub-0.19.4-py3-none-any.whl.metadata (14 kB)
  Obtaining dependency information for regex!=2019.12.17 from https://files.pythonhosted.org/packages/8f/3e/4b8b40eb3c80aeaf360f0361d956d129bb3d23b2a3ecbe3a04a8f3bdd6d3/regex-2023.10.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Using cached regex-2023.10.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (40 kB)
  Obtaining dependency information for tokenizers<0.19,>=0.

In [2]:
import pandas as pd

df_test = pd.read_csv('review_dataset.csv')
df_test.head()

,review_id,user_id,business_id,stars,useful,funny,cool,text,date,review_links,label
0,wsFRDsHxz2mM_Ettgn1qQg,x8ErSBur0SsnL1lZwP5o4Q,qf4LecJDQWIt0gt6VJWFPw,4,0,0,0,We got diverted to Tampa and decided to make t...,2017-06-28 01:07:04,https://www.yelp.com/biz/qf4LecJDQWIt0gt6VJWFP...,0.0
1,Fg6B-QCi9IE9NfUQufFfNQ,tYVgGWTevie6UTMyINC3kw,M2ZrrqseHE5xssUrOL0Gtg,5,1,0,0,"Quick& friendly service, great portion size fo...",2015-12-20 22:15:00,https://www.yelp.com/biz/M2ZrrqseHE5xssUrOL0Gt...,0.0
2,L3MXURl5Uh5AGHbIeln14g,5pY-0Jw_qHxOgLeU3iyaGw,aKOdfQcjDG2NQtIEmq21ew,5,0,0,0,Being from Cincinnati we know ipas and urban s...,2018-06-26 19:12:30,https://www.yelp.com/biz/aKOdfQcjDG2NQtIEmq21e...,0.0
3,8UwvPwoMzoyJIV6Wd5hZbg,157pNZAX0otHcMi0doqmqA,a7FSs8soBoxfkPXvzSsvbg,4,0,0,0,Swung by here while I was in town and was so h...,2011-07-28 13:32:47,https://www.yelp.com/biz/a7FSs8soBoxfkPXvzSsvb...,0.0
4,LSV_FqBhazt_R4q-JOqbdQ,nnXNbPjckCCFUE5JwnQj0A,SheKIt6Z_h-yzbkU8yjoPA,5,0,0,0,Nothing has changed since my last review. Thei...,2017-09-20 15:09:28,https://www.yelp.com/biz/SheKIt6Z_h-yzbkU8yjoP...,0.0


In [9]:
df_test.label.value_counts()

label
1.0    4300
0.0     700
Name: count, dtype: int64

In [10]:
from ClassifierFinal import ClassifierModel
import torch
from torch.utils.data import DataLoader,TensorDataset
from transformers import BertTokenizer
import pandas as pd
import numpy as np

test_texts = np.array(df_test['text'])
test_texts_list = test_texts.tolist()
test_labels = np.array(df_test['label'])

In [11]:
test_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
inputs = test_tokenizer(test_texts_list, padding=True, truncation=True, return_tensors='pt')

In [12]:
test_labels = torch.tensor(test_labels)
test_dataset = TensorDataset(inputs['input_ids'], inputs['attention_mask'],test_labels)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=True)

In [13]:
model = ClassifierModel()
model.load_state_dict(torch.load('classifier.pth'))

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


<All keys matched successfully>

In [14]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

ClassifierModel(
  (bert_model): BertForSequenceClassification(
    (bert): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(30522, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0-11): 12 x BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutput(
                (dense): Linear(in_features=768, out_feature

<h2> Using Positive Unlabelled Data

In [21]:
from sklearn.metrics import f1_score,recall_score,precision_score

model.eval()
test_predictions = []
true_labels = []
threshold = 0.35

with torch.no_grad():
    for test_batch in  test_loader:
        
        test_input_ids, test_attention_mask, test_labels = test_batch
        test_input_ids, test_attention_mask, test_labels = test_input_ids.to(device), test_attention_mask.to(device), test_labels.to(device)
        test_outputs = model(test_input_ids, attention_mask=test_attention_mask)
        predicted_class = (test_outputs[:,1] > threshold).float()
        test_predictions.extend(predicted_class.cpu().numpy())
        true_labels.extend(test_labels.cpu().numpy())

test_f1 = f1_score(true_labels, test_predictions)
test_recall = recall_score(true_labels, test_predictions)
test_precision = precision_score(true_labels, test_predictions)
print(f'Test F1 Score: {test_f1:.4f}, Test Recall: {test_recall:.4f}, Test Precision: {test_precision:.4f}')

Test F1 Score: 0.8131, Test Recall: 0.7674, Test Precision: 0.8646


In [22]:
from sklearn.metrics import confusion_matrix

conf_matrix = confusion_matrix(true_labels,test_predictions )
print("Confusion Matrix:")
print(conf_matrix)

Confusion Matrix:
[[ 183  517]
 [1000 3300]]


In [17]:
np.sum(np.array(test_predictions) == 0)

1183

In [20]:
print("Number of Genuine:"+str(np.sum(np.array(test_predictions) == 1)))
print("Number of Fake:"+str(np.sum(np.array(test_predictions) == 0)))

Number of Genuine:3817
Number of Fake:1183


<h2> Test on Unseen Restaurant Reviews Dataset

In [33]:
df_test2 = pd.read_csv('restaurant_reviews_anonymized.csv',encoding='latin-1')
df_test2.Real.value_counts()

Real
0    55
1    55
Name: count, dtype: int64

In [34]:
test_texts2 = np.array(df_test2['Review'])
test_labels2 = np.array(df_test2['Real'])
test_texts_list2 = test_texts2.tolist()
inputs = test_tokenizer(test_texts_list2, padding=True, truncation=True, return_tensors='pt')
labels2 = torch.tensor(test_labels2)
test_dataset2 = TensorDataset(inputs['input_ids'], inputs['attention_mask'], labels2)
test_loader2 = DataLoader(test_dataset2, batch_size=1, shuffle=True)

In [35]:
model.eval()
test_predictions = []
true_labels = []

threshold = 0.35 #Adjust the threshold as needed

with torch.no_grad():
    for test_batch in  test_loader2:
        
        test_input_ids, test_attention_mask, test_labels = test_batch
        test_input_ids, test_attention_mask, test_labels = test_input_ids.to(device), test_attention_mask.to(device), test_labels.to(device)
        test_outputs = model(test_input_ids, attention_mask=test_attention_mask)
        predicted_class = (test_outputs[:,1] > threshold).float()
        test_predictions.extend(predicted_class.cpu().numpy())
        true_labels.extend(test_labels.cpu().numpy())

test_f1 = f1_score(true_labels, test_predictions)
test_recall = recall_score(true_labels, test_predictions)
test_precision = precision_score(true_labels, test_predictions)
print(f'Test F1 Score: {test_f1:.4f}, Test Recall: {test_recall:.4f}, Test Precision: {test_precision:.4f}')

Test F1 Score: 0.5440, Test Recall: 0.6182, Test Precision: 0.4857


In [36]:
conf_matrix = confusion_matrix(true_labels,test_predictions )
print("Confusion Matrix:")
print(conf_matrix)

Confusion Matrix:
[[19 36]
 [21 34]]


In [37]:
print("Number of Genuine:"+str(np.sum(np.array(test_predictions) == 1)))
print("Number of Fake:"+str(np.sum(np.array(test_predictions) == 0)))

Number of Genuine:70
Number of Fake:40


<h2> Test on Unseen Hotel Review Data (Cross Domain)

In [39]:
df_test3 = pd.read_csv('deceptive-opinion.csv')
label_mapping = {'deceptive': 0, 'truthful': 1}
df_test3['deceptive'] = df_test3['deceptive'].map(label_mapping)
df_test3.deceptive.value_counts()

deceptive
1    800
0    800
Name: count, dtype: int64

In [41]:
test_texts3 = np.array(df_test3['text'])
test_labels3 = np.array(df_test3['deceptive'])
test_texts_list3 = test_texts3.tolist()
inputs = test_tokenizer(test_texts_list3, padding=True, truncation=True, return_tensors='pt')
labels3 = torch.tensor(test_labels3)
test_dataset3 = TensorDataset(inputs['input_ids'], inputs['attention_mask'], labels3)
test_loader3 = DataLoader(test_dataset3, batch_size=1, shuffle=True)

In [42]:
model.eval()
test_predictions = []
true_labels = []

threshold = 0.35 #Adjust the threshold as needed

with torch.no_grad():
    for test_batch in  test_loader3:
        
        test_input_ids, test_attention_mask, test_labels = test_batch
        test_input_ids, test_attention_mask, test_labels = test_input_ids.to(device), test_attention_mask.to(device), test_labels.to(device)
        test_outputs = model(test_input_ids, attention_mask=test_attention_mask)
        predicted_class = (test_outputs[:,1] > threshold).float()
        test_predictions.extend(predicted_class.cpu().numpy())
        true_labels.extend(test_labels.cpu().numpy())

test_f1 = f1_score(true_labels, test_predictions)
test_recall = recall_score(true_labels, test_predictions)
test_precision = precision_score(true_labels, test_predictions)
print(f'Test F1 Score: {test_f1:.4f}, Test Recall: {test_recall:.4f}, Test Precision: {test_precision:.4f}')

Test F1 Score: 0.6057, Test Recall: 0.7863, Test Precision: 0.4926


In [43]:
conf_matrix = confusion_matrix(true_labels,test_predictions )
print("Confusion Matrix:")
print(conf_matrix)

Confusion Matrix:
[[152 648]
 [171 629]]


In [44]:
print("Number of Genuine:"+str(np.sum(np.array(test_predictions) == 1)))
print("Number of Fake:"+str(np.sum(np.array(test_predictions) == 0)))

Number of Genuine:1277
Number of Fake:323
